# Goal: generating some matchings and attempt to code up Gale-Shapley 

In [1]:
# manually produce a ranking system for both sides 

# first case:
girl_rank1 = {
    "1": ["a", "c", "b", "d"],
    "2": ["d","c","b","a"],
    "3": ["b","a","d","c"],
    "4": ["c","d","a","b"]
}
boy_rank1 = {
    "a": ["3","4","2","1"],
    "b": ["1","4","2","3"],
    "c": ["2","3","1","4"],
    "d": ["3","1","2","4"]
}

# second case:
girl_rank2 = {"A": ["D","E","F"], "B": ["D","E","F"], "C": ["D","E","F"]}
boy_rank2 = {"D": ["B","A","C"], "E": ["B","A","C"], "F": ["A","B","C"]}

# third case:
boy_rank3 = {"X": ["A","B","C"], "Y": ["B", "A", "C"], "Z": ["A","B","C"]}
girl_rank3 = {"A": ["Y","X","Z"], "B": ["X","Y","Z"], "C": ["X","Y","Z"]}

#fourth case:
boy_rank4 = {
    "A": ["O","M","N","L","P"],
    "B": ["P","N","M", "L","O"],
    "C": ["M", "P", "L", "O","N"],
    "D": ["P","M","O","N","L"],
    "E": ["O","L","M","N","P"]
}
girl_rank4 = {
    "L": ["D","B","E","C","A"],
    "M": ["B","A","D","C","E"],
    "N": ["A","C","E","D","B"],
    "O": ["D","A","C","B","E"],
    "P": ["B","E","A","C","D"]
}



In [2]:
# Redo of Gale-Shapley algorithm - I understood it wrong 

def gale_shapley11(proposers_rank, receivers_rank):
    #basically doing deep copy so we dont modify the original rankings
    proposers = {key: value[:] for key, value in proposers_rank.items()}  # Copying the lists in proposers_rank
    receivers = {key: value[:] for key, value in receivers_rank.items()}  # Copying the lists in receivers_rank
    unmatched = {"boy": [i for i in proposers]}
    engagement = {} # (girl: boy)
    day = 0
    rejects = {boy: [] for boy in proposers}
    while len(unmatched["boy"]) > 0: #until every man is engaged 
        propositions = {}
        print("engagement: ", engagement, "unmatched: ", unmatched)
        # round 1: boy's turn to propose 
        for unmatched_boy in unmatched["boy"]:
            highest_girl = proposers[unmatched_boy].pop(0)
            # if already rejected before, then go down the list
            while highest_girl in rejects[unmatched_boy]: 
                highest_girl = proposers[unmatched_boy].pop(0)
            if highest_girl not in propositions:
                propositions[highest_girl] = [unmatched_boy]
            else:
                propositions[highest_girl].append(unmatched_boy)
        print("propositions: ", propositions)
        # now we have a list of propositions
        # round 2: girl's turn to choose 
        for proposed_girl in propositions:
            considerations = receivers[proposed_girl].copy() #temporary copy 
            highest_boy = considerations.pop(0) 
            while highest_boy not in propositions[proposed_girl]: #going down the list
                #to the highest-ranked proposing boy 
                highest_boy = considerations.pop(0) 
            # assume we have a match 
            if proposed_girl not in engagement: #the girl not engaged - then matched
                engagement[proposed_girl] = highest_boy
                unmatched["boy"].remove(highest_boy) #remove from bachelor pool
            else: #if engaged, then consider
                current = engagement[proposed_girl]
                alternative = highest_boy
                ranking = receivers[proposed_girl]
                if ranking.index(alternative) < ranking.index(current): 
                    #if the alternative is better: break up with current engagement 
                    # broken-with boy is back in the market 
                    unmatched["boy"].append(current)
                    print(proposed_girl, "broke up with", current, "for", alternative)
                    rejects[current].append(proposed_girl)
                    unmatched["boy"].remove(alternative) #remove from bachelor pool
                    engagement[proposed_girl] = alternative
                else:
                    print(proposed_girl, "rejects", alternative, "and stays with", current) 
                    rejects[alternative].append(proposed_girl)
            day += 1
    print("final engagement: ", engagement, "it took", day, "days")
    # print("rejects: ", rejects)


gale_shapley11(boy_rank4, girl_rank4) #boys proposing

print("NOW we do girl proposing")

gale_shapley11(girl_rank4, boy_rank4) #girls proposing 


engagement:  {} unmatched:  {'boy': ['A', 'B', 'C', 'D', 'E']}
propositions:  {'O': ['A', 'E'], 'P': ['B', 'D'], 'M': ['C']}
engagement:  {'O': 'A', 'P': 'B', 'M': 'C'} unmatched:  {'boy': ['D', 'E']}
propositions:  {'M': ['D'], 'L': ['E']}
M broke up with C for D
engagement:  {'O': 'A', 'P': 'B', 'M': 'D', 'L': 'E'} unmatched:  {'boy': ['C']}
propositions:  {'P': ['C']}
P rejects C and stays with B
engagement:  {'O': 'A', 'P': 'B', 'M': 'D', 'L': 'E'} unmatched:  {'boy': ['C']}
propositions:  {'L': ['C']}
L rejects C and stays with E
engagement:  {'O': 'A', 'P': 'B', 'M': 'D', 'L': 'E'} unmatched:  {'boy': ['C']}
propositions:  {'O': ['C']}
O rejects C and stays with A
engagement:  {'O': 'A', 'P': 'B', 'M': 'D', 'L': 'E'} unmatched:  {'boy': ['C']}
propositions:  {'N': ['C']}
final engagement:  {'O': 'A', 'P': 'B', 'M': 'D', 'L': 'E', 'N': 'C'} it took 9 days
NOW we do girl proposing
engagement:  {} unmatched:  {'boy': ['L', 'M', 'N', 'O', 'P']}
propositions:  {'D': ['L', 'O'], 'B': [

In [3]:
# # # let's attempt to code up a 1-to-1 matching 

# # I THINK THIS IS THE BOSTON MATCHING ALGORITHM! 


# # # reverse the ranking so that we have a stack for each 
# # for girl in girl_rank:
# #     girl_rank[girl].reverse()
# # for boy in boy_rank:
# #     boy_rank[boy].reverse()

# print("girl rank", girl_rank)
# print("boy rank", boy_rank)
# #now we will attempt to code up a matching system
# matches = [] #format: (girl, boy)

# no_days = 3

# #list of unmatched people
# unmatched = {"girl": [i for i in girl_rank],"boy": [i for i in boy_rank]}

# day = 0 
# while len(unmatched["girl"]) + len(unmatched["boy"]) > 0: #while still unmatched people
#     propositions = {}
#     print(boy_rank, girl_rank)
#     # S1: boy's turn to propose 
#     for unmatched_boy in unmatched["boy"]:
#         highest_girl = boy_rank[unmatched_boy].pop()
#         while highest_girl not in unmatched["girl"]: #going down to list
#             #to the first unmatched girl 
#             highest_girl = boy_rank[unmatched_boy].pop()
#         propositions[unmatched_boy] = highest_girl
#     print("propositions", propositions)
#     # now we have a list of propositions of boy to their current highested-ranked girls 
#     # S2: girl's turn to accept 
#     for unmatched_girl in unmatched["girl"]: #for each unmatched girl 
#         highest_boy = girl_rank[unmatched_girl].pop()
#         while highest_boy not in unmatched["boy"]: #going down the list
#             #to the first unmatched girl 
#             highest_boy = girl_rank[unmatched_girl].pop()
#         if highest_boy in propositions and propositions[highest_boy] == unmatched_girl: #if the boy they want is proposing
#             a_match = (propositions[highest_boy], highest_boy)
#             matches.append(a_match)
#             print("found", a_match)
#             unmatched["girl"].remove(a_match[0])
#             unmatched["boy"].remove(a_match[1])
#             del boy_rank[highest_boy]
#             del girl_rank[propositions[highest_boy]]
#     # S3: restore the ranking for both if they are not matched
#     for proposition in propositions: #restore for the boys
#         if propositions[proposition] in unmatched["girl"]: #still unmatched
#             boy_rank[proposition].append(propositions[proposition])
#     # print("unmatched", unmatched)
#     day += 1
#     # now we have a list of propositions

# print("stable matches found:", matches)
# print("number of days needed", day)
